In [1]:
import pandas as pd 
import numpy as np
import math

In [2]:
import networkx as nx

In [3]:
import math

In [4]:
df = pd.read_csv("../data/data_yrqt_cl_bt.csv")

In [5]:
df.head(30)

,Lender_ID,Borrower_ID,Amount,Status,Lender_Rate,Borrower_Rate,Grade,Time,Quater,Year,loan_classification,borrower_type
0,267,4578,1000.0,Paid,0.0400,0.0400,AA,2005-11-15 00:00:00,4,2005,1,1
1,267,60169,1000.0,Paid,0.1200,0.1700,E,2005-11-18 00:00:00,4,2005,1,1
2,267,1663,2500.0,Paid,0.2500,0.2500,HR,2005-11-22 00:00:00,4,2005,1,1
3,4037,75787,1000.0,Paid,0.1300,0.1300,AA,2005-11-22 00:00:00,4,2005,1,1
4,267,75787,1000.0,Paid,0.1300,0.1300,AA,2005-11-22 00:00:00,4,2005,1,1
5,7386,62475,250.0,Paid,0.0435,0.0435,AA,2005-11-23 00:00:00,4,2005,1,1
6,267,62475,1000.0,Paid,0.0435,0.0435,AA,2005-11-23 00:00:00,4,2005,1,1
7,1506,60541,2000.0,Paid,0.1200,0.1200,C,2005-11-25 00:00:00,4,2005,1,1
8,267,9015,4800.0,Paid,0.0900,0.0900,NC,2005-11-25 00:00:00,4,2005,1,1
9,7385,60541,50.0,Paid,0.1200,0.1200,C,2005-11-25 00:00:00,4,2005,1,1


In [6]:
df_sorted = df.sort_values(by=['Time'])

In [7]:
df.shape

(3394979, 12)

In [8]:
def SocialCredit_formula(lam, p, g, l, h):
    par_1 = lam/(1-lam)
    par_2 = (lam*p + (1-lam))/(lam+(1-lam)*p)
    par_3 = (lam + (1-lam)*p)/(lam*p +(1-lam))
    sc = 1/(1+math.pow(par_1,g)+math.pow(par_2,l)+math.pow(par_3,h))
    return sc

In [9]:
def SocialCredit_calculator(target_table,lam, p):
    sc_list = []
    idx = target_table.index
    min_idx = min(idx)
    for i in idx:
        if i == min_idx:
            g = 0
            temp_brwt = target_table.loc[:i,:]
            temp_brwt = temp_brwt.borrower_type.value_counts()
            l = temp_brwt.get(key = -1)
            h = temp_brwt.get(key = 1)
            if l == None:
                l = 0                
            if h == None:
                h =0               
            sc = SocialCredit_formula(lam, p, g, l, h)
            sc_list.append(sc)
        else:
            if target_table.borrower_type[i] == 1:
                g = 1
                temp_brwt = target_table.loc[:i,:]
                temp_brwt = temp_brwt.borrower_type.value_counts()
                l = temp_brwt.get(key = -1)
                h = temp_brwt.get(key = 1)
                if l == None:
                    l = 0                
                if h == None:
                    h =0                   
                sc = SocialCredit_formula(lam, p, g, l, h)
                sc_list.append(sc)
            else: 
                g = -1
                temp_brwt = target_table.loc[:i,:]
                temp_brwt = temp_brwt.borrower_type.value_counts()
                l = temp_brwt.get(key = -1)
                h = temp_brwt.get(key = 1)
                if l == None:
                    l = 0                
                if h == None:
                    h =0   
                sc = SocialCredit_formula(lam, p, g, l, h)
                sc_list.append(sc)
    return pd.Series(sc_list, index=idx, name='social_credit')      

In [10]:
bwr_list = list(set(df_sorted.Borrower_ID))

In [11]:
df_sorted['social_credit'] = 999

In [12]:
c=0
for bwr in bwr_list:
    c += 1
    if c%1000 == 0:
        print(c)
        
    temp = df_sorted[df_sorted.Borrower_ID == bwr]
    sc_temp = SocialCredit_calculator(temp,0.4, 0.39)
    df_sorted.update(sc_temp)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000


In [13]:
df_sorted.head()

,Lender_ID,Borrower_ID,Amount,Status,Lender_Rate,Borrower_Rate,Grade,Time,Quater,Year,loan_classification,borrower_type,social_credit
0,267,4578,1000.0,Paid,0.04,0.04,AA,2005-11-15 00:00:00,4,2005,1,1,0.26051
1,267,60169,1000.0,Paid,0.12,0.17,E,2005-11-18 00:00:00,4,2005,1,1,0.26051
2,267,1663,2500.0,Paid,0.25,0.25,HR,2005-11-22 00:00:00,4,2005,1,1,0.26051
3,4037,75787,1000.0,Paid,0.13,0.13,AA,2005-11-22 00:00:00,4,2005,1,1,0.26051
4,267,75787,1000.0,Paid,0.13,0.13,AA,2005-11-22 00:00:00,4,2005,1,1,0.29674


In [17]:
df_sorted[df_sorted.Borrower_ID == 1663]

,Lender_ID,Borrower_ID,Amount,Status,Lender_Rate,Borrower_Rate,Grade,Time,Quater,Year,loan_classification,borrower_type,social_credit
2,267,1663,2500.0,Paid,0.25,0.25,HR,2005-11-22 00:00:00,4,2005,1,1,0.26051
